In [1]:
import os 
import pandas 

In [4]:
def downloadfile(downloadpath, url):
    import requests
    # 發送 HTTP 請求，獲取 XML 資料
    response = requests.get(url)
    
    # 如果請求成功（狀態碼為 200），則寫入檔案
    if response.status_code == 200:
        with open(downloadpath, 'wb') as file:
            file.write(response.content)
        print(f"XML 檔案已成功下載到 {downloadpath}")
    else:
        print(f"請求失敗，狀態碼：{response.status_code}")

In [ ]:


# 使用方法
url = "https://tisvcloud.freeway.gov.tw/history/motc20/VD.xml"
downloadpath = "VD.xml"  # 可以根據需要修改下載的路徑和檔案名稱
downloadfile(downloadpath, url)


XML 檔案已成功下載到 VD.xml
